In [1]:
import cv2
import numpy as np
from os.path import join
import os
import matplotlib.pyplot as plt
import random
import datetime
import pandas as pd
# placeholder path
dir_RK = r"C:\Users\roma0\cvpr_group\lab_3\images\RK_dataset\test"
dir_RB = r"C:\Users\roma0\cvpr_group\lab_3\images\RB_dataset\test"
dir_AS = r"C:\Users\roma0\cvpr_group\lab_3\images\AS_dataset\test"
dir_NO = r"C:\Users\roma0\cvpr_group\lab_3\images\without"
dir_test_RK = r"C:\Users\roma0\cvpr_group\lab_3\images\test_RK"
dir_test_RB = r"C:\Users\roma0\cvpr_group\lab_3\images\test_RB"
dir_test_AS = r"C:\Users\roma0\cvpr_group\lab_3\images\test_AS"
dir_test_NO = r"C:\Users\roma0\cvpr_group\lab_3\images\test_without"

In [2]:
def extractor(fn):
  im = cv2.imread(fn,0)
  im = cv2.resize(im, (im.shape[0]//4, im.shape[1]//4), interpolation = cv2.INTER_AREA)
  return extract.compute(im, detect.detect(im))[1]

def form_voc(path, number):
  k = 0
  for filename in os.listdir(path):
      k=k+1
      bow_kmeans_trainer.add(extractor(os.path.join(path, filename)))
      if k == number:
        break  

def form_train_set(number):
    traindata, trainlabels = [],[]

    k = 0
    for filename in os.listdir(dir_RK):
        k = k+1
        try:
            traindata.extend(bow_features(os.path.join(dir_RK, filename))); trainlabels.append(1)
        except:
            continue
        if k == number:
            break
    k = 0
    for filename in os.listdir(dir_RB):
        k = k+1
        try:
            traindata.extend(bow_features(os.path.join(dir_RB, filename))); trainlabels.append(2)
        except:
            continue
        if k == number:
            break
    k = 0
    for filename in os.listdir(dir_AS):
        k = k+1
        try:
            traindata.extend(bow_features(os.path.join(dir_AS, filename))); trainlabels.append(3)
        except:
            continue
        if k == number:
            break  
    k = 0
    for filename in os.listdir(dir_NO):
        k = k+1
        try:
            traindata.extend(bow_features(os.path.join(dir_NO, filename))); trainlabels.append(4)
        except:
            continue
        if k == number:
            break
    return traindata, trainlabels  

    
def bow_features(fn):
  im = cv2.imread(fn,0)
  im = cv2.resize(im, (im.shape[0]//4, im.shape[1]//4), interpolation = cv2.INTER_AREA)
  return extract_bow.compute(im, detect.detect(im))

def predict(fn):
  f = bow_features(fn);  
  p = svm.predict(f)
  return p
def test_obj(path, metric, Id):
    for filename in os.listdir(path):
        metric[0].append(filename)
        img = cv2.imread(os.path.join(path, filename), 0) 
        img = cv2.resize(img, (img.shape[0]//4, img.shape[1]//4), interpolation = cv2.INTER_AREA)
        start_time = datetime.datetime.now()
        img_predict = predict(os.path.join(path, filename))
        end_time = datetime.datetime.now()
        time_diff = (end_time - start_time)
        time = time_diff.total_seconds()
        metric[2].append(time)
        if (img_predict[1][0][0] != Id):
            metric[1].append(1)
        else: 
            metric[1].append(0) 
        plt.imshow(img),plt.show()
        print(img_predict[1][0][0])
        print()
   
def test_noobj(path, metric, Id):
    for filename in os.listdir(path):
        metric[0].append(filename)
        img = cv2.imread(os.path.join(path, filename), 0)
        img = cv2.resize(img, (img.shape[0]//4, img.shape[1]//4), interpolation = cv2.INTER_AREA)
        start_time = datetime.datetime.now()
        img_predict = predict(os.path.join(path, filename))
        end_time = datetime.datetime.now()
        time_diff = (end_time - start_time)
        time = time_diff.total_seconds()
        metric[2].append(time)
        if (img_predict[1][0][0] != 4):
            metric[1].append(1)
        else: 
            metric[1].append(0)    
        plt.imshow(img),plt.show()
        print(img_predict[1][0][0])
        print()
 

In [ ]:

detect = cv2.xfeatures2d.SURF_create()
extract = cv2.xfeatures2d.SURF_create()
#detect = cv2.AKAZE_create(descriptor_type = 2)
#extract = cv2.AKAZE_create(descriptor_type = 2)
flann_params = dict(algorithm = 1, trees = 5)
matcher = cv2.FlannBasedMatcher(flann_params, {})

bow_kmeans_trainer = cv2.BOWKMeansTrainer(40)
extract_bow = cv2.BOWImgDescriptorExtractor(extract, matcher)

form_voc(dir_RK, 20)
form_voc(dir_RB, 20)
form_voc(dir_AS, 20)
form_voc(dir_NO, 20)

voc = bow_kmeans_trainer.cluster()
extract_bow.setVocabulary(voc)

traindata, trainlabels = form_train_set(40)
print(traindata)
print(trainlabels)

c = list(zip(traindata,trainlabels))
random.shuffle(c)
traindata, trainlabels = zip(*c)

print(traindata)
print(trainlabels)

svm = cv2.ml.SVM_create()
svm.setKernel(2)
svm.setType(100)
svm.trainAuto(np.array(traindata), cv2.ml.ROW_SAMPLE, np.array(trainlabels))



In [ ]:
import datetime
import pandas as pd
metric_shoe = [[],[],[]]
metric_bird = [[],[],[]]
metric_disk = [[],[],[]]
metric_noobj = [[],[],[]]
test_obj(dir_test_RK, metric_shoe, 1)
test_obj(dir_test_RB, metric_disk, 2)
test_obj(dir_test_AS, metric_bird, 3)
test_noobj(dir_test_NO, metric_noobj, 4)  

df1 = pd.DataFrame({"filename " : metric_shoe[0], "eror I": metric_shoe[1], "time": metric_shoe[2]})
df2 = pd.DataFrame({"filename " : metric_disk[0], "eror I": metric_disk[1], "time": metric_disk[2]})
df3 = pd.DataFrame({"filename " : metric_bird[0], "eror I": metric_bird[1], "time": metric_bird[2]})
df4 = pd.DataFrame({"filename " : metric_noobj[0], "eror II": metric_noobj[1], "time": metric_noobj[2]})

df1.to_csv('RK_dataset_SURF.csv') 
df2.to_csv('RB_dataset_SURF.csv')
df3.to_csv('AS_dataset_SURF.csv')
df4.to_csv('Noobj_dataset_SURF.csv') 
print(metric_shoe)
print(metric_disk)
print(metric_bird)
print(metric_noobj)

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
capture = cv2.VideoCapture("test2.mp4")
#fourcc = cv2.VideoWriter_fourcc('M','P','4','V')
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
fps = 20
w, h = 720, 480
stream = cv2.VideoWriter("bird_surf.avi", fourcc, fps, (w, h))

while capture.isOpened():
    ret, frame = capture.read()
    if ret == False:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
    frame_gray = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2RGB)
    #cv2.rectangle(frame_gray, (w//2,h//2 ), (50, 50), (150, 0, 255), 2)
    #cv2.line(frame_gray, (0, 10), (w-10, 10), (250, 0, 0), 2)
    f = extract_bow.compute(frame_gray, detect.detect(frame_gray))
    try:
        img_predict = svm.predict(f)
    except:
        cv2.putText(frame_gray,'No features',(10,60), font, 3, (0,255,0),1,cv2.LINE_AA)
        stream.write(frame_gray)
        cv2.imshow('frame2', frame_gray)
        continue
    if (img_predict[1][0][0] == 3.0):
      cv2.putText(frame_gray,'+',(10,60), font, 3, (0,255,0),1,cv2.LINE_AA)
    else:
      cv2.putText(frame_gray,'-',(10,60), font, 3, (0,0,255),1,cv2.LINE_AA)
    stream.write(frame_gray)
    cv2.imshow('frame2', frame_gray)

    if cv2.waitKey(25) == ord('q'):
        break
capture.release()
stream.release()
cv2.destroyAllWindows()
